In [1]:
#------------------------------------------------------
# Name: CarLanesDetection
#
# Author: Mohsin Karim
# Date Created 12/15/2016
# Last Modified 12/15/2016
#-------------------------------------------------------

In [2]:
#importing libraries
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
import sys
import io
import collections
%matplotlib inline 

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [3]:
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)


In [4]:
def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)


In [5]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


In [6]:
frame_l = []
frame_r = []
frame_lSlope = []
frame_rSlope = []


def draw_lines(img, lines, color=[0, 0, 255], thickness=10):
    """
    Calculating slope, checking is /0 than move without furthure calculations
    """
    
    # list for last 5 slopes for each left and right lanes
    left_slopes = []
    right_slopes = []
    averageL = 0
    averageR = 0
    
    #max_records for slopes to hold
    max_records_slopes = 10
    records_counter_L = 0
    records_counter_R = 0
    
    
    leftlanes = []
    rightlanes = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if (x2-x1) != 0:
                #calculate Slope
                slope = (y2-y1)/(x2-x1)
                if np.absolute(slope) > 0.5: #tested different values and this is the best with least shake in lines
                    if slope < 0: #left lanes
                            if records_counter_L > max_records_slopes:
                                left_slopes.pop(0)
                                records_counter_L -=1;
                            else:
                                records_counter_L +=1;
                                
                            left_slopes.append(slope)
                            averageL = np.sum(left_slopes) + (slope - np.sum(left_slopes)) / records_counter_L
                            if np.absolute(averageL - slope) < 0.1 :
                                leftlanes.append((x1,y1))
                                leftlanes.append((x2,y2))
                    else:
                        if records_counter_R > max_records_slopes:
                                right_slopes.pop(0)
                                records_counter_R -=1;
                        else:
                                records_counter_R +=1;

                        right_slopes.append(slope)
                        averageR = np.sum(right_slopes) + (slope - np.sum(right_slopes)) / records_counter_R
                      
                        if np.absolute(averageR - slope) < 0.1 :
                            rightlanes.append((x1,y1))
                            rightlanes.append((x2,y2))               
    
    #cv2 fitline 0.01 would be a good default value for reps and aeps based on documentation
    [vx,vy,x,y] = cv2.fitLine(np.array(leftlanes, dtype=np.int32), cv2.DIST_FAIR,0,0.01,0.01)
    [vx_r,vy_r,x_r,y_r] = cv2.fitLine(np.array(rightlanes, dtype=np.int32), cv2.DIST_FAIR,0,0.01,0.01)    
    slopeL = vy/vx
    slopeR = vy_r / vx_r

    leftX = y - (slopeL*x)
    rightY = y_r - (slopeR*x_r)
    
    frame_l.append(leftX)
    frame_r.append(rightY)
    frame_lSlope.append(slopeL)
    frame_rSlope.append(slopeR)

    
    # Average out the lines and slope
    left = np.sum(frame_l)/len(frame_l) #avg[0]
    lefts = np.sum(frame_lSlope)/len(frame_lSlope) #avg[1]
    right = np.sum(frame_r)/len(frame_r) #avg[2]
    rights = np.sum(frame_rSlope)/len(frame_rSlope) #avg[3]

    lTop = (int(img.shape[0]/1.6) - left) / lefts
    lBottom = int((img.shape[0] - left) / lefts)

    rTop = (int(img.shape[0]/1.6) - right) / rights
    rBottom = int((img.shape[0] - right) / rights)

    #Drawing the left and right lanes
    cv2.line(img, (int(lBottom), img.shape[0]), (int(lTop), int(img.shape[0]/1.6)), color, thickness)
    cv2.line(img, (int(rBottom), img.shape[0]), (int(rTop), int(img.shape[0]/1.6)), color, thickness)

In [7]:
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img


In [8]:
# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)


In [9]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


##### Main processing pipeline ####

The following processes are performed in the this function

1. Image is conveted to HSV
2. Color range is defined for white color lanes
3. Color range is defined for yellow color lanes
4. Yellow color is extracted from the image using inRange 
5. White color is extracted from the image using inRange
6. The two images extracted yellow and extracted white is combined.
7. Applied canny on the combined image
8. Reduced the selection to the desired region.
9. Hough trasform on the region.

In [10]:
#MAIN Processing Code using HSV

def DetectLanesHSV(original_image):
    
    img_dimensions = original_image.shape
    #print('This image is:', type(original_image), 'with dimesions:', original_image.shape)
    
    hsv = cv2.cvtColor(original_image, cv2.COLOR_BGR2HSV)
    
    sensitivity = 40
    color_ranges_white = [([0,0,255-sensitivity],[255,sensitivity,255]) ]
    color_ranges = [([5,50,50],[25,255,255]) ]
    color_ranges_yellow = [([5,100,100],[25,255,255])]
    
    output_yellow = None
    output_white = None
    
    #storing image dimensions
    img_dimensions = original_image.shape
    gray = grayscale(original_image) #Getting and storing the orginal image in grayscale
    
    kernel_size  = 7
    low_threshold = 70
    high_threshold = 200
    
    
    #Define Rectanle Points
    vertices = np.array([[(0,img_dimensions[0]),
                          (img_dimensions[1] * ((1.6)/3),img_dimensions[0]/1.71),
                          (img_dimensions[1] * ((1.6)/3),img_dimensions[0]/1.71),
                          (img_dimensions[1],img_dimensions[0])]],dtype=np.int32)
      
    rho = 1
    theta = np.pi/180
    threshold = 10
    min_line_len = 10
    max_line_gap = 30
    line_image = np.copy(original_image)
    
    
    #Getting Yellow lines from image
    for (lower, upper) in color_ranges_yellow :
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        mask = cv2.inRange(hsv,lower,upper)
        output_yellow = cv2.bitwise_and(original_image,original_image,mask = mask)
    
    #Getting White lines from image
    for (lower, upper) in color_ranges_white :
        lower = np.array(lower, dtype = "uint8")
        upper = np.array(upper, dtype = "uint8")
        mask = cv2.inRange(hsv,lower,upper)
        output_white = cv2.bitwise_and(original_image,original_image,mask = mask)

    #combine yellow and white
    yellow_white = output_yellow + output_white
    canny_img = canny(yellow_white,low_threshold,high_threshold)
    region = region_of_interest(canny_img,vertices)
    h_lines = hough_lines(region, rho, theta, threshold, min_line_len, max_line_gap)
    final_image = weighted_img(h_lines,original_image)
    
    return final_image

In [11]:
#files = os.listdir("Challange_frames/")
#for file in files:
#    #reading in an image
#    original_image = cv2.imread('Challange_frames/' + file)
    #Function Call    
#    final_image = DetectLanesHSV(original_image)
#    cv2.imwrite(r'C:\Users\mohsin\Desktop\CarND-LaneLines-P1\CC\\' + file ,final_image)
    
cap = cv2.VideoCapture("challenge.mp4")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH);
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT); 
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('challengeHSV_processed.mp4',fourcc, 15.0, (int(width),int(height)))

while(True):
    # Capture frames
    ret, frame = cap.read()
    if ret:
        f_image = DetectLanesHSV(frame)
        # write the frame
        out.write(f_image)
    else :
        break
cap.release()
out.release()
cv2.destroyAllWindows()
print ("Finished....")


Finished....
